In [ ]:
#DO NOT RUN IF YOU ARE NOT USING WINDOWS AND/OR NOT USING ANACONDA
%pip install numpy
%pip install pandas
%pip install os-sys
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install "tensorflow-gpu<2.10"
%pip install "tensorflow<2.10"
%pip install "keras<2.10"

In [ ]:
# If you are on Google Colab run this
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D
import sklearn.model_selection

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

In [2]:
def data_collection():
    # create directory normalized if it does not exist
    os.makedirs('./normalized', exist_ok=True) 

    # import csv file
    # ON GOOGLE COLAB
        # read csv file from your google drive. find the file in your drive and copy the path and replace
        # the path in the read_csv function with the path to your file
    df = pd.read_csv('FallAllD2.csv')
    # convert all columns to float32
    df = df.astype('float32')

    return df

In [3]:
def run_all():
    df = data_collection()
    df = df[df['Device'] == 2]
    for i in range(0,7):
        x_train, x_test, y_train, y_test = data_split(df, i)
        print(x_train.shape, y_train.shape)
        print(x_test.shape, y_test.shape)

        # Create and Train Model
        model = model_create(x_train)
        print("Model Created")
        model = train_and_accurary_model(model, x_train, x_test, y_train, y_test)
        print("Model Trained")

In [ ]:
def data_label(df):
    # add a new column called "IsFall" that is 1 if the ActivityID > 100, and 0 if it is not
    df['IsFall'] = df['ActivityID'].apply(lambda x: 1 if x > 100 else 0)
    return df

def data_split(df, num):
    df = data_label(df)
    # split the data into features and labels
    if num == 0: x = df[['Device','Acc_x','Acc_y','Acc_z', 'Gyr_x', 'Gyr_y', 'Gyr_z', 'Bar_x', 'Bar_y']]
    elif num == 1: x = df[['Device','Acc_x','Acc_y','Acc_z', 'Gyr_x', 'Gyr_y', 'Gyr_z']]
    elif num == 2: x = df[['Device','Gyr_x', 'Gyr_y', 'Gyr_z', 'Bar_x', 'Bar_y']]
    elif num == 3: x = df[['Device','Acc_x','Acc_y','Acc_z', 'Bar_x', 'Bar_y']]
    elif num == 4: x = df[['Device','Acc_x','Acc_y','Acc_z']]
    elif num == 5: x = df[['Device','Gyr_x', 'Gyr_y', 'Gyr_z']]
    elif num == 6: x = df[['Device','Bar_x', 'Bar_y']]
    y = df['IsFall']
    
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    x = x.reshape((x.shape[0], 1, x.shape[1]))
    
    #Spliting Data
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y,test_size = 0.2)
    print('x y shape: ', x_train.shape, y_train.shape)

    # the following is not needed, the reshape is already done
    if False:
        # reshape training and testing data
        y_train = np.array(y_train).reshape(-1,1) # (-1,1) because our data has a single feature, and a 'n' amount of rows
        y_test = np.array(y_test).reshape(-1,1) # (-1,1) bec  ause our data has a single feature, and a 'n' amount of rows
        #reshape the features for the LSTM layer (I REALLY WANT TO FIX UGHHHG)
        steps = x_test.shape[1]
        x_train = np.array(x_train).reshape(x_train.shape[0], steps, 1)
        x_test = np.array(x_test).reshape(x_test.shape[0], steps, 1)

    return x_train, x_test, y_train, y_test

In [ ]:
from sklearn.metrics import confusion_matrix
def model_create(x_train):
  model = Sequential()
  model.add(LSTM(512, input_shape=(x_train.shape[1], x_train.shape[2])))
  model.add(Dropout(0.2))
  
  # add a Flatten layer using x_train as input shape
  #model.add(Flatten(input_shape=x_train.shape[1:]))

  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))

  # for activity classification, we need 136 neurons in the output layer and categorical crossentropy as the loss function
  #model.add(Dense(136, activation='softmax'))
  #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  # for IsFall classification, we need 1 neuron in the output layer and binary crossentropy as the loss function
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def train_and_accurary_model(model, x_train, x_test, y_train, y_test):

  # train the model
  model.fit(x_train, y_train, epochs=10, batch_size=256, validation_split=0.1)

  # evaluate the model
  test_loss, test_Acc = model.evaluate(x_test, y_test)
  print('Test accuracy:', test_Acc)
  model.summary()
  print("Confusion Matrix")
  y_pred = model.predict(x_test)
  y_pred = (y_pred > 0.5)
  confusion_mtx = confusion_matrix(y_test, y_pred)
  confusion_mtx_percent = confusion_mtx / confusion_mtx.sum(axis=1)[:, np.newaxis]
  print(confusion_mtx_percent)

  # Save Model to Local Machine
  file_name = 'model' + str(i) + '_device2_aggregate.h5'

  model.save(file_name)
  from google.colab import files
  files.download(file_name)  # Download to local machine
  print("Model downloaded to local machine")

  # Save Model to Google Drive
  model.save('/content/drive/My Drive/' + file_name )
  print("Model saved to Google Drive")
  
  return model


In [ ]:
# Get model from folder all_models. take model summaries and put them into a txt file. download txt file to local machine
def get_model_summaries():
    import os
    from keras.models import load_model
    from keras.utils.vis_utils import plot_model
    from keras.utils import print_summary
    
    # get all files in the folder
    files = os.listdir('all models')
    print(files)
    
    # open a file to write the model summaries to
    f = open("model_summaries.txt", "w")
    
    # loop through all files in the folder
    for file in files:
        # load the model
        model = load_model('all_models/' + file)
        # get the model summary
        model.summary(print_fn=lambda x: f.write(x + '\n'))
        # get the model plot
        plot_model(model, to_file='model_plots/' + file + '.png', show_shapes=True, show_layer_names=True)
        # write a line break
        f.write('\n\n')
    f.close()
    

Functions that are not used

In [17]:
def get_activity(num = 0):
    # a dictionary that maps the case number (ActivityID) with stirng label
    activity_dict = {
    101: 'Fall F, walking, trip',
    102: 'Fall F, walking, trip, rec.',
    103: 'Fall F, walking, slip',
    104: 'Fall F, walking, slip, rec.',
    105: 'Fall F, walking, slip, rot.',
    106: 'Fall F, walking, slip, rot., rec.',
    107: 'Fall B, walking, slip',
    108: 'Fall B, walking, slip, rec.',
    109: 'Fall B, walking, slip, rot.',
    110: 'Fall B, walking, slip rot., rec.',
    111: 'Fall F, walking, syncope',
    112: 'Fall B, walking, syncope',
    113: 'Fall L, walking, syncope',
    114: 'Fall, syncope, table',
    115: 'Fall F, try sit',
    116: 'Fall F, try sit, rec.',
    117: 'Fall B, try sit',
    118: 'Fall B, try sit, rec.',
    119: 'Fall L, try sit',
    120: 'Fall L, try sit, rec.',
    121: 'Fall F, jog, trip',
    122: 'Fall F, jog, trip, rec.',
    123: 'Fall F, jog, slip',
    124: 'Fall F, jog, slip, rev.',
    125: 'Fall F, jog, slip, rot.',
    126: 'Fall F, jog, slip, rot., rec.',
    127: 'Fall L, bed',
    128: 'Fall L, bed, rec.',
    129: 'Fall F, chair, syncope',
    130: 'Fall B, chair, syncope',
    131: 'Fall L, chair, syncope',
    132: 'Fall F, syncope',
    133: 'Fall B, syncope',
    134: 'Fall L, syncope',
    135: 'Fall, syncope, slide over a wall',
    1: 'Start clap hands',
    2: 'Clap hands',
    3: 'Stop clap hands',
    4: 'Clap hands 1',
    5: 'Start wave hands',
    6: 'wave hands',
    7: 'Stop wave hands',
    8: 'Raising hand up',
    9: 'Moving hand down',
    10: 'Move hand up -> down',
    11: 'Hand shaking',
    12: 'Beating a table',
    13: 'Sitting down',
    14: 'Standing up',
    15: 'Fail to stand up',
    16: 'Lying down',
    17: 'Turning while lying',
    18: 'Rising up',
    19: 'Start walking',
    20: 'Walking slowly',
    21: 'Stop walking',
    22: 'Walking quickly',
    23: 'Stumbling',
    24: 'Jogging slowly',
    25: 'Jogging quickly',
    26: 'Jumping slightly',
    27: 'Jumping strongly',
    28: 'B...'
    }
    return activity_dict[num]

In [3]:
#Debugging function
def data_plot(df):
    #debugging function. does not need to be called, or return anything

    # plot the first 100 column 'AccelerationX' and 'AccelerationX_fft'
    df[['Acc'[0]], ['Acc'[1]], ['Acc'[2]]].iloc[:100].plot()

    # Versus Head (the same)
    #df[['AccelerationX_fft', 'AccelerationY_fft', 'AccelerationZ_fft']].head(100).plot()
    

In [ ]:
def data_normalize(df): #Not in use
    # for each column in 'Accerlation', 'AccelerationY', 'AccelerationZ',
    # add a new column that is the fft of the original column
    for c in ['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ']:
        df[c + '_fft'] = np.fft.fft(df[c])
        
    return df

def data_categorical(df): #Not in use
    # convert column 'Device' to numerical data
    df['Device'] = df['Device'].astype('category')
    df['Device'] = df['Device'].cat.codes

    return df